# Análise de Modelos

Em exemplos anteriores usamos todo o dataset para treinar nossos modelos. Contudo, o objetivo de um modelo é lidar com novos dados e a partir dele fazer previsões. Dessa forma, se usarmos os mesmos dados para treino e avaliações o modelo mostrará resultados muito otimistas. Portanto, a ideia e treina-lo com uma parte do dataset e usar a outra para testes.

## Table of Contents

*   [1. Carregando Biblioteca e Dataset](#cell1)
*   [2. Avaliação de Modelos](#cell2)

### Carregando Biblioteca e Dataset

Vamos primeiramente carregar os dados e ferramentas que precisamos.

In [1]:
# Carrega tidyverse
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
# Instalar pacotes rlang e tidymodels
install.packages("rlang")
install.packages("tidymodels")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependencies ‘listenv’, ‘parallelly’, ‘cachem’, ‘future’, ‘warp’, ‘lhs’, ‘memoise’, ‘DiceDesign’, ‘vctrs’, ‘lifecycle’, ‘tidyselect’, ‘pillar’, ‘patchwork’, ‘generics’, ‘globals’, ‘hardhat’, ‘ipred’, ‘furrr’, ‘slider’, ‘ellipsis’, ‘cpp11’, ‘GPfit’, ‘broom’, ‘cli’, ‘conflicted’, ‘dials’, ‘dplyr’, ‘ggplot2’, ‘infer’, ‘modeldata’, ‘parsnip’, ‘recipes’, ‘rsample’, ‘rstudioapi’, ‘tibble’, ‘tidyr’, ‘tune’, ‘workflows’, ‘workflowsets’, ‘yardstick’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [ ]:
# Biblioteca para modelos
library(tidymodels)

# Carrega tidyverse
library(tidyverse)

A empresa IBM, mantém em seu site uma série de conjunto de dados. Nesse caso usaremos o <a href="https://developer.ibm.com/exchanges/data/all/airline/">Airline Dataset</a> que contém dados de mais de 200 milhões de voos domésticos feitos em território americano, coletados pelo Departamento de Estatísticas de Transporte dos Estados Unidos. Por questões didáticas usaremos a amostra desses dados e não o arquivo inteiro.

<ul>
    <li>data source: <a href="https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz" target="_blank">https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz</a></li>
    <li>data type: csv</li>
</ul>

In [ ]:
# url onde os dados estão localizados
# url <- "https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz"

# download do arquivo
#download.file(url, destfile = "lax_to_jfk.tar.gz")

# extrair arquivo 
# remova tar = "internal se estiver rodando este programa localmente" 
# untar("lax_to_jfk.tar.gz", tar = "internal")

# Lemos o arquivo csv => read_csv (Se for de um formato diferente use a função apropriada, verifique a documentação de readr)
companhias_aereas <- read_csv("dados/losAngeles_para_novaYork.csv",
                     col_types = cols(
                      'DivDistance' = col_number(),
                      'DivArrDelay' = col_number()
                      ))

### Avaliação de Modelos

Usaremos a biblioteca tidymodels para lidar com aprendizado de máquina, construir dados para treino ou teste e mais. Lembrando da pergunta que estamos tentando responder: Podemos prever atrasos na chegada de um voo? Vamos usar o dataset atrasos para construir nosso modelo.

In [ ]:
# Atrasos é um subconjunto do dataset companhias_aereas.
# Removemos os valores NA
# E selecionamos as colunas que nos interessam (Atrasos causados pelo tempo (WeatherDelay), Mês(Month)...)
atrasos <- companhias_aereas %>% 
    replace_na(list(CarrierDelay = 0,
                    WeatherDelay = 0,
                    NASDelay = 0,
                    SecurityDelay = 0,
                    LateAircraftDelay = 0)) %>%
    select(c(ArrDelayMinutes, DepDelayMinutes, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, DayOfWeek, Month))
atrasos

Para construir um modelo é preciso primeiro dividí-lo em um dataset de treino e outro de teste. Então, usa-se a função `set.seed()`para garantir que os subconjuntos gerados a partir do dataset serão sempre os mesmos toda vez que esse código é executado. Observando que por padrão esta função divide 75% dos dados para treino e os 25% restante para teste.

In [ ]:
set.seed(1234)
subconjunto <- initial_split(atrasos) 
treino <- training(subconjunto)
teste <- testing(subconjunto)

Uma vez divido o dataset especifica-se o modelo a ser usado nele. Nesse exemplo, usou-se a regressão linear, a variável atrasos na partida (DepDelayMinutes) para prever atrasos na chegada (ArrDelayMinutes) e treino,por sua vez, é usado para treinar o modelo. O código abaixo demonstra como fazer isso.

In [ ]:
# Use regressão linear
rl <- linear_reg() %>%
  set_engine(engine = "lm")

# Exiba a função
rl

In [ ]:
modelo_ajustado <- rl_modelo %>% 
    # Usa-se fit para ajustar o modelo especificado
    fit(ArrDelayMinutes ~ DepDelayMinutes, data = treino)

modelo_ajustado 

A fim de verificar algumas predições do modelo ajustado usa-se a função `predict()`. Ela produz uma coluna `.pred` com os valores de atrasos na chegada. Nesse caso, ainda não usou-se os dados de teste e sim os de treino. O dataframe criado é salvo na variável resultados_treino.

In [ ]:
resultados_treino <- modelo_ajustado %>%
  # Faca previsoes e salve os valores
  predict(new_data = treino) %>%
  # Crie uma nova coluna originais com do treino
  mutate(originais = treino$ArrDelayMinutes)

head(resultados_treino)

Para testar o modelo só é preciso trocar a variável treino por teste.

In [ ]:
resultados_teste <- modelo_ajustado %>%
  predict(new_data = teste) %>%
  mutate(originais = teste$ArrDelayMinutes)

head(resultados_teste)

Para verificar a validade desse modelo, pode-se usar as métricas EQM ou o coeficiente de determinação mencionados anteriormente. Ao invés de aplicar manualmente essas fórmulas é melhor usar as funções `rmse()` e `rsq()`

In [ ]:
# Calcula a raiz do erro quadrátrico médio dos dados obtidos a partir do treino
rmse(resultados_treino, truth = originais,
     estimate = .pred)

In [ ]:
# Calcula a raiz do erro quadrátrico médio dos dados obtidos a partir do teste
rmse(resultados_teste, truth = originais,
     estimate = .pred)

In [ ]:
rsq(resultados_treino, truth = originais,
    estimate = .pred)

In [ ]:
rsq(resultados_teste, truth = originais,
    estimate = .pred)